# <center>编程练习部分</center>

## 1.句子生成器

In [1]:
import random
class SentenceGenerator(object):
    def __init__(self, gram_str=None, line_split='\n', target_split='=', cell_split="|"):
        if gram_str is not None:
            self.set_gram(gram_str, line_split, target_split, cell_split)
    
    @staticmethod
    def format_gram(gram_str, line_split='\n', target_split='=', cell_split="|"):
        """把输入的语法字符串格式化为字典对象，以便后续操作"""
        gram = {}
        for line in gram_str.split(line_split):
            if line.strip():
                target, cells_str = line.split(target_split)
                cells = cells_str.split(cell_split)
                gram[target.strip()] = [cell.strip().split() for cell in cells]
        return gram
    
    @classmethod
    def sentence_gen(cls, gram=None, target=None):
        """根据语法字典和句子标签，生成句子"""
        if gram is None:
            return ''
        if target not in gram:
            return target
        # 递归
        expaned = [cls.sentence_gen(gram, t) for t in random.choice(gram[target])]
        sentence = ''.join(x if x != '/n' else '\n' for x in expaned if x != 'null')
        return sentence
    
    def set_gram(self, gram_str, line_split='\n', target_split='=', cell_split="|"):
        """设置对象的语法， 输入语法字符串"""
        self.gram = self.format_gram(gram_str, line_split, target_split, cell_split)
    
    def get_sentence(self, target):
        """根据句子标签，获取生成的句子"""
        return self.sentence_gen(self.gram, target)
    
    def get_sentences(self, target, n):
        """根据句子标签以及数量，获取生成的句子列表"""
        return [self.get_sentence(target) for _ in range(n)]

语法1

In [2]:
gram1 = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""

In [3]:
gen1 = SentenceGenerator(gram1, target_split='=')

In [4]:
gen1.get_sentence('human')

'俺想找点乐子'

In [5]:
gen1.get_sentences('human', 10)

['我看看玩的',
 '俺找找乐子',
 '我想找点玩的',
 '我们找找玩的',
 '我们找找乐子',
 '我们看看乐子',
 '我想找点玩的',
 '我找找玩的',
 '我想找点乐子',
 '我们看看乐子']

## 2. 使用新数据源完成语言模型的训练(使用自己爬取的段子)

In [6]:
import pandas as pd
import numpy as np
import re
import jieba
from collections import Counter
from operator import itemgetter

import waduanzi.waduanzi.orm as orm    # 数据库orm模型

### 获取数据

In [7]:
dbase = orm.SqlOrm()

In [8]:
query = dbase.session.query(orm.Joke)

In [9]:
query.count()

60926

In [10]:
for i in range(10):
    print(query[i].to_dict())

{'id': '217358', 'title': '平时负责后勤工作', 'content': '平时负责后勤工作。今天检查员工宿舍卫生情况，发现厨师空调没关，就顺手关了，然后去和厨师说了下，下次离开宿舍前空调要关了。结果中午吃排骨，厨师给我打一碗大骨头（基本没肉），我看其他同事全是肉…', 'likes': 983, 'unlikes': -14, 'url': 'http://www.waduanzi.com/archives/217358'}
{'id': '217359', 'title': '公司抽奖，我和一女同事抽到连着的电影票', 'content': '公司抽奖，我和一女同事抽到连着的电影票。', 'likes': 1356, 'unlikes': -6, 'url': 'http://www.waduanzi.com/archives/217359'}
{'id': '217360', 'title': '在街上一哥们过来问我，兄弟要不要手机', 'content': '在街上一哥们过来问我，兄弟要不要手机？我说：什么型号的，几成新？他深吸一口烟，缓缓的开口说道：', 'likes': 1196, 'unlikes': -16, 'url': 'http://www.waduanzi.com/archives/217360'}
{'id': '217361', 'title': '一室友友趁有钱男友出差和网友幽 会，结果发现中招了，', 'content': '一室友友趁有钱男友出差和网友幽会，结果发现中招了，无奈约了自己闺蜜陪去的医院。', 'likes': 1336, 'unlikes': -13, 'url': 'http://www.waduanzi.com/archives/217361'}
{'id': '217362', 'title': '本人女', 'content': '本人女，34岁，家庭主妇一枚，现求工作一份，非正当不做！最好照顾孩子两不误，求各位帅哥美女给过了！最好是新乡市里的！', 'likes': 1056, 'unlikes': -11, 'url': 'http://www.waduanzi.com/archives/217362'}
{'id': '217363', 'title': '楼主洒水车司机', 'c

In [11]:
df = pd.DataFrame([info.to_dict() for info in query.all()])

In [12]:
df.head()

,content,id,likes,title,unlikes,url
0,平时负责后勤工作。今天检查员工宿舍卫生情况，发现厨师空调没关，就顺手关了，然后去和厨师说了下...,217358,983,平时负责后勤工作,-14,http://www.waduanzi.com/archives/217358
1,公司抽奖，我和一女同事抽到连着的电影票。,217359,1356,公司抽奖，我和一女同事抽到连着的电影票,-6,http://www.waduanzi.com/archives/217359
2,在街上一哥们过来问我，兄弟要不要手机？我说：什么型号的，几成新？他深吸一口烟，缓缓的开口说道：,217360,1196,在街上一哥们过来问我，兄弟要不要手机,-16,http://www.waduanzi.com/archives/217360
3,一室友友趁有钱男友出差和网友幽会，结果发现中招了，无奈约了自己闺蜜陪去的医院。,217361,1336,一室友友趁有钱男友出差和网友幽 会，结果发现中招了，,-13,http://www.waduanzi.com/archives/217361
4,本人女，34岁，家庭主妇一枚，现求工作一份，非正当不做！最好照顾孩子两不误，求各位帅哥美女给...,217362,1056,本人女,-11,http://www.waduanzi.com/archives/217362


In [13]:
jokes_ori = df['content']

In [14]:
jokes_ori[:10]

0    平时负责后勤工作。今天检查员工宿舍卫生情况，发现厨师空调没关，就顺手关了，然后去和厨师说了下...
1                                 公司抽奖，我和一女同事抽到连着的电影票。
2      在街上一哥们过来问我，兄弟要不要手机？我说：什么型号的，几成新？他深吸一口烟，缓缓的开口说道：
3              一室友友趁有钱男友出差和网友幽会，结果发现中招了，无奈约了自己闺蜜陪去的医院。
4    本人女，34岁，家庭主妇一枚，现求工作一份，非正当不做！最好照顾孩子两不误，求各位帅哥美女给...
5    楼主洒水车司机，今天正工作呢，看见一男子骑着电驴子载个女的，不走侧道走主道，恰恰公路两边是很...
6                 高中一学弟，打球摔断了左手和右腿，所以每天他妈妈推着轮椅来送他上下学，。
7    买个房子还要什么单身证明！你是瞧不起单身啊，还要证明昭告天下？信不信我叫两声吓死你！wao！...
8    我去我哥家，刚坐下，还没冲茶，我嫂子在厨房问：“老公，你要柠檬味的，还是姜汁味的？要热的还是...
9                                            一美女来公司送货。
Name: content, dtype: object

### 清洗数据

In [15]:
clean_data = lambda x:  re.findall(r'\w+', x)

In [16]:
joke_test = clean_data(jokes_ori[0])

In [17]:
joke_test

['平时负责后勤工作',
 '今天检查员工宿舍卫生情况',
 '发现厨师空调没关',
 '就顺手关了',
 '然后去和厨师说了下',
 '下次离开宿舍前空调要关了',
 '结果中午吃排骨',
 '厨师给我打一碗大骨头',
 '基本没肉',
 '我看其他同事全是肉']

In [18]:
jokes = [clean_data(str) for str in jokes_ori]

In [19]:
jokes[:10]

[['平时负责后勤工作',
  '今天检查员工宿舍卫生情况',
  '发现厨师空调没关',
  '就顺手关了',
  '然后去和厨师说了下',
  '下次离开宿舍前空调要关了',
  '结果中午吃排骨',
  '厨师给我打一碗大骨头',
  '基本没肉',
  '我看其他同事全是肉'],
 ['公司抽奖', '我和一女同事抽到连着的电影票'],
 ['在街上一哥们过来问我', '兄弟要不要手机', '我说', '什么型号的', '几成新', '他深吸一口烟', '缓缓的开口说道'],
 ['一室友友趁有钱男友出差和网友幽会', '结果发现中招了', '无奈约了自己闺蜜陪去的医院'],
 ['本人女',
  '34岁',
  '家庭主妇一枚',
  '现求工作一份',
  '非正当不做',
  '最好照顾孩子两不误',
  '求各位帅哥美女给过了',
  '最好是新乡市里的'],
 ['楼主洒水车司机',
  '今天正工作呢',
  '看见一男子骑着电驴子载个女的',
  '不走侧道走主道',
  '恰恰公路两边是很长的那种花坛',
  '男的停下电车',
  '一人翻花坛跑了',
  '女的双手抱头等着被淋',
  '我只能把水停了',
  '顺便告诉她',
  '妹啊',
  '找个像我这样的好人再嫁一次吧',
  '看着妹子那感激的眼神',
  '第一次这么痛恨自己为嘛是女人'],
 ['高中一学弟', '打球摔断了左手和右腿', '所以每天他妈妈推着轮椅来送他上下学'],
 ['买个房子还要什么单身证明', '你是瞧不起单身啊', '还要证明昭告天下', '信不信我叫两声吓死你', 'wao', 'wao', 'waowu'],
 ['我去我哥家', '刚坐下', '还没冲茶', '我嫂子在厨房问', '老公', '你要柠檬味的', '还是姜汁味的', '要热的还是冷的'],
 ['一美女来公司送货']]

### 切词获取胞词库

In [20]:
cutor = lambda x: jieba.lcut(x)

In [21]:
joke_cut_test = cutor(''.join(joke_test))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shredder\AppData\Local\Temp\jieba.cache
Loading model cost 0.576 seconds.
Prefix dict has been built succesfully.


In [22]:
joke_cut_test

['平时',
 '负责',
 '后勤工作',
 '今天',
 '检查',
 '员工',
 '宿舍',
 '卫生',
 '情况',
 '发现',
 '厨师',
 '空调',
 '没关',
 '就',
 '顺手',
 '关',
 '了',
 '然后',
 '去',
 '和',
 '厨师',
 '说',
 '了',
 '下',
 '下次',
 '离开',
 '宿舍',
 '前',
 '空调',
 '要关',
 '了',
 '结果',
 '中午',
 '吃',
 '排骨',
 '厨师',
 '给',
 '我',
 '打',
 '一碗',
 '大',
 '骨头',
 '基本',
 '没肉',
 '我',
 '看',
 '其他',
 '同事',
 '全是',
 '肉']

In [23]:
corpus = []

In [24]:
for i, joke in enumerate(jokes):
    corpus += cutor(''.join(joke))
    if i % 1000 == 0: 
        print("已处理了%d个" % i)

已处理了0个
已处理了1000个
已处理了2000个
已处理了3000个
已处理了4000个
已处理了5000个
已处理了6000个
已处理了7000个
已处理了8000个
已处理了9000个
已处理了10000个
已处理了11000个
已处理了12000个
已处理了13000个
已处理了14000个
已处理了15000个
已处理了16000个
已处理了17000个
已处理了18000个
已处理了19000个
已处理了20000个
已处理了21000个
已处理了22000个
已处理了23000个
已处理了24000个
已处理了25000个
已处理了26000个
已处理了27000个
已处理了28000个
已处理了29000个
已处理了30000个
已处理了31000个
已处理了32000个
已处理了33000个
已处理了34000个
已处理了35000个
已处理了36000个
已处理了37000个
已处理了38000个
已处理了39000个
已处理了40000个
已处理了41000个
已处理了42000个
已处理了43000个
已处理了44000个
已处理了45000个
已处理了46000个
已处理了47000个
已处理了48000个
已处理了49000个
已处理了50000个
已处理了51000个
已处理了52000个
已处理了53000个
已处理了54000个
已处理了55000个
已处理了56000个
已处理了57000个
已处理了58000个
已处理了59000个
已处理了60000个


In [25]:
len(corpus)

2464276

In [26]:
count = Counter(corpus)

In [27]:
count.most_common(50)

[('的', 107835),
 ('我', 92394),
 ('了', 91924),
 ('你', 41096),
 ('说', 34940),
 ('是', 27615),
 ('在', 25913),
 ('就', 25372),
 ('他', 24381),
 ('她', 18079),
 ('都', 16512),
 ('不', 15921),
 ('一个', 15878),
 ('有', 15256),
 ('去', 13681),
 ('给', 12205),
 ('着', 11525),
 ('和', 10945),
 ('一', 10862),
 ('也', 9998),
 ('啊', 9871),
 ('把', 9837),
 ('上', 9481),
 ('问', 9349),
 ('人', 9079),
 ('到', 9042),
 ('还', 8575),
 ('看', 8223),
 ('这', 8050),
 ('要', 7979),
 ('那', 7411),
 ('对', 7357),
 ('没', 7286),
 ('我们', 7047),
 ('什么', 7024),
 ('来', 6887),
 ('吃', 6731),
 ('又', 6656),
 ('老婆', 6652),
 ('很', 6612),
 ('好', 6559),
 ('时候', 6488),
 ('个', 6436),
 ('吗', 6358),
 ('想', 6316),
 ('吧', 6216),
 ('今天', 5966),
 ('被', 5940),
 ('跟', 5854),
 ('然后', 5722)]

### 使用2-gram预测

In [28]:
corpus_2 = [corpus[i] + corpus[i+1] for i in range(len(corpus) - 1)]

In [29]:
corpus_2[:50]

['平时负责',
 '负责后勤工作',
 '后勤工作今天',
 '今天检查',
 '检查员工',
 '员工宿舍',
 '宿舍卫生',
 '卫生情况',
 '情况发现',
 '发现厨师',
 '厨师空调',
 '空调没关',
 '没关就',
 '就顺手',
 '顺手关',
 '关了',
 '了然后',
 '然后去',
 '去和',
 '和厨师',
 '厨师说',
 '说了',
 '了下',
 '下下次',
 '下次离开',
 '离开宿舍',
 '宿舍前',
 '前空调',
 '空调要关',
 '要关了',
 '了结果',
 '结果中午',
 '中午吃',
 '吃排骨',
 '排骨厨师',
 '厨师给',
 '给我',
 '我打',
 '打一碗',
 '一碗大',
 '大骨头',
 '骨头基本',
 '基本没肉',
 '没肉我',
 '我看',
 '看其他',
 '其他同事',
 '同事全是',
 '全是肉',
 '肉公司']

In [30]:
count_2 = Counter(corpus_2)

In [31]:
count_2.most_common(20)

[('了我', 5987),
 ('的时候', 5426),
 ('我的', 4629),
 ('我说', 3971),
 ('说我', 3781),
 ('给我', 3202),
 ('我就', 3149),
 ('说你', 3033),
 ('你的', 2528),
 ('的人', 2205),
 ('的说', 2193),
 ('都是', 2149),
 ('的我', 2114),
 ('我是', 1891),
 ('了个', 1877),
 ('了一个', 1861),
 ('问我', 1805),
 ('不知道', 1745),
 ('的是', 1738),
 ('来了', 1712)]

In [32]:
corpus_2_len = len(corpus_2)

In [33]:
corpus_2_len

2464275

In [34]:
count_2_len = len(count_2)

In [35]:
count_2_len

946634

In [36]:
def prob_2(sentence):
    cells = cutor(''.join(clean_data(sentence)))
    sen_pro = 1
    for i, cell in enumerate(cells[:-1]):
        cell_n = cells[i+1]
        cell_pro = (count_2[cell+cell_n] + 1) / (corpus_2_len + count_2_len)
        sen_pro *= cell_pro
    return sen_pro

In [37]:
prob_2("大妈跳广场舞")

9.313685389065057e-17

In [38]:
prob_2("美女跳广场舞")

3.1045617963550187e-17

### 判断生成句子合理性

In [108]:
gram = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个|那个|此
noun =>   女人 |  篮球 | 桌子 | 小猫|二哈|大爷|小明
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的|漂亮的|洋气的
"""
generator = SentenceGenerator(gram, target_split='=>')

In [109]:
sentences = generator.get_sentences('sentence', 10)
sentences

['此好看的篮球听着此桌子',
 '那个洋气的漂亮的蓝色的小明看着一个蓝色的二哈',
 '一个大爷听着此洋气的蓝色的小明',
 '那个女人坐在这个漂亮的桌子',
 '这个二哈看着此大爷',
 '那个女人听着这个洋气的漂亮的二哈',
 '一个二哈听着这个大爷',
 '此篮球坐在一个蓝色的蓝色的桌子',
 '此桌子看见一个漂亮的篮球',
 '那个洋气的漂亮的蓝色的小小的小小的小小的好看的洋气的桌子看见此女人']

In [110]:
def get_best_sentence(sentences):
    pros = [(sen, prob_2(sen)) for sen in sentences]
    return(sorted(pros, key=itemgetter(1), reverse=True))
        

In [111]:
get_best_sentence(sentences)

[('这个二哈看着此大爷', 7.387873798496053e-27),
 ('一个二哈听着这个大爷', 1.1436239916121822e-29),
 ('此桌子看见一个漂亮的篮球', 2.3731277972034352e-33),
 ('那个女人坐在这个漂亮的桌子', 8.470422619388833e-34),
 ('此好看的篮球听着此桌子', 3.9319047001891e-42),
 ('此篮球坐在一个蓝色的蓝色的桌子', 2.9473566801752426e-48),
 ('那个女人听着这个洋气的漂亮的二哈', 2.3375067078343082e-48),
 ('一个大爷听着此洋气的蓝色的小明', 7.74487057938285e-52),
 ('那个洋气的漂亮的蓝色的小明看着一个蓝色的二哈', 1.9401740862533755e-68),
 ('那个洋气的漂亮的蓝色的小小的小小的小小的好看的洋气的桌子看见此女人', 5.1058295530804514e-101)]

## 总结

语料库来自6万多个小段子，其语境应该比较贴合生活和网络用语，最后合理性判断的结果一般，除了知道语料库不大以外，还需要进一步分析优化算法。